# desilike

## Código para especificar likelihoods de DESI

### Tutorial #1: likelihoods, emuladores, samplers, profilers, Fisher

# Metas
Al final de este tutorial sabrás:
- Cómo escribir likelihoods de clustering
- Cómo emular su teoría
- Cómo correr inferencia (sampling, profiling, Fisher)


# Environment

Para ejecutar los siguientes ejemplos de desilike, sólo necesitas tener instalado **desilike** (el cual requiere **cosmoprimo**) con:
```
python -m pip install git+https://github.com/cosmodesi/desilike#egg=desilike
```
Para el solucionador analítico:
```
python -m pip install jax[cpu]
```
Sin embargo, la manera más fácil es usar el environment de cosmodesi en NERSC:
```
source /global/common/software/desi/users/adematti/cosmodesi_environment.sh main  # cargar el environment
${COSMODESIMODULES}/install_jupyter_kernel.sh main  # Esto sólo es necesario hacerlo una vez
```
NB: Para remover el kernel anterior:
```
rm -rf ${HOME}/.local/share/jupyter/kernels/cosmodesi-main
```

# Material

Este tutorial

[desilike docs](https://desilike.readthedocs.io/), en particular [getting started](https://desilike.readthedocs.io/en/latest/user/getting_started.html)

[desilike notebooks](https://github.com/cosmodesi/desilike/tree/main/nb)

Aplicaciones prácticas: Ejemplos de los beneficios de MPI!

[Aplicación al mock challenge (velocileptors, pybird, folps)](https://github.com/cosmodesi/desi-y1-kp7/tree/main/mock_challenge_y1)

[Aplicación de inferenica cosmológica con los mocks de Y1](https://github.com/cosmodesi/desi-y1-kp7/tree/main/mock_y1)

[Aplicación a los forecasts de DESI Y5/Y1](https://github.com/cosmodesi/desilike/tree/main/nb) (Uendert Andrade)

[Aplicación a la validación del blinding](https://github.com/cosmodesi/desi-y1-kp45/tree/main/blinding) (Uendert Andrade & Edmond Chaussidon for fnl)
 
[Aplicación a las pruebas de las colisiones de fibras](https://github.com/mathildepinon/desi_fiber_collisions) (Mathilde Pinon)

# Agradecimientos

[Contribuciones (directas o indirectas) de muchas personas](https://github.com/cosmodesi/desilike/tree/main#acknowledgments)

# Clustering likelihood

Describamos cómo especificar el likelihood para los multipolos de un espectro de potencias o una función de correlación 
- template
- teoría
- observable

## Template

Cómo se parametriza el espectro de potencias lineal como input de los códigos de teoría. Varias opciones son posibles, entre ellas:

### Parametrización estándar (como en BOSS/eBOSS) 
en términos de:
- parámetros de escala $q_{\parallel}$, $q_{\perp}$
- variación del growth rate of structure $df = f / f^{\mathrm{fid}}$

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import jax; jax.config.update('jax_platform_name', 'cpu')
%matplotlib inline

In [ ]:
from desilike.theories.galaxy_clustering import\
StandardPowerSpectrumTemplate

template = StandardPowerSpectrumTemplate(z=1.1)
# Esto muestra todos los parámetros, después de que hayan sido inicializados
print(template.all_params)

Si se quiere $q_{\parallel}$, $q_{\perp}$ en términos del parámetro isotrópico $q_{\mathrm{iso}}$ y de escalamiento AP $q_{\mathrm{ap}}$

In [ ]:
template = StandardPowerSpectrumTemplate(z=1.1, apmode='qisoqap')
print(template.all_params)

### Parametrización [ShapeFit](https://arxiv.org/abs/2106.07641)
En términos de:
- Parámetros de escala $q_{\parallel}$, $q_{\perp}$
- Variación en el growth rate of structure $df = f / f^{\mathrm{fid}}$
- Parámetro de tilt de ShapeFit $dm$

In [ ]:
from desilike.theories.galaxy_clustering import\
ShapeFitPowerSpectrumTemplate

template = ShapeFitPowerSpectrumTemplate(z=1.1)
# Esto muestra todos los parámetros, una vez hayan sido inicializados
print(template.all_params)

In [ ]:
import numpy as np
from matplotlib import colors, cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import pyplot as plt

from desilike.theories.galaxy_clustering import  KaiserTracerPowerSpectrumMultipoles, ShapeFitPowerSpectrumTemplate, WiggleSplitPowerSpectrumTemplate, BandVelocityPowerSpectrumTemplate

z, b1 = 1.1, 2.

def get_theory(template):
    template.init.update({'z': z, 'fiducial': 'DESI'})
    theory = KaiserTracerPowerSpectrumMultipoles(template=template)
    theory.params['b1'].update(fixed=False, value=b1)
    theory.params['sn0'].update(fixed=False, value=0.)
    return theory

n = 5
cmap = plt.get_cmap('jet', n)

def plot(template, template_name, params):
    theory_compression = get_theory(template=template)
    theory_compression()
    center_params = theory_compression.runtime_info.pipeline.input_values
    fig, lax = plt.subplots(1, len(params), sharex=False, sharey=True, figsize=(4 * len(params), 4))
    lax = lax.flatten()
    fig.subplots_adjust(wspace=0.25)
    
    for iax, (param, values) in enumerate(params.items()):
        theory_compression(**center_params)
        ax = lax[iax]
        for ivalue, value in enumerate(values):
            poles = theory_compression(**{param: value})
            for ill, ell in enumerate(theory_compression.ells):
                ax.plot(theory_compression.k, theory_compression.k * poles[ill], color=cmap(ivalue))
        ax.set_xlabel(r'$k$ [$h/\mathrm{Mpc}$]')
        ax.set_ylabel(r'$k P_{\ell}(k)$ [$(\mathrm{Mpc}/h)^{2}$]')
        ax.grid(True)
        norm = colors.Normalize(vmin=values[0], vmax=values[-1])
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        divider = make_axes_locatable(ax)
        cax = divider.new_vertical(size='5%', pad = 0.5)
        fig.add_axes(cax)
        cbar = plt.colorbar(sm, cax=cax, ax=ax, orientation='horizontal')
        cbar.set_label(theory_compression.all_params[param].latex(inline=True), rotation=0)
    plt.show()

In [ ]:
plot(ShapeFitPowerSpectrumTemplate(), 'shapefit',\
{'df': np.linspace(0.9, 1.1, n), 'dm': np.linspace(-0.1, 0.1, n), 'qpar': np.linspace(0.98, 1.02, n), 'qper': np.linspace(0.98, 1.02, n)})

### Parametrización Directa

En términos de los parámetros cosmológicos base

In [ ]:
from desilike.theories.galaxy_clustering import\
DirectPowerSpectrumTemplate

template = DirectPowerSpectrumTemplate(z=1.1)
# Esto muestra todos los parámetros, una vez haya sido inicializado el calculador
print(template.all_params)

In [ ]:
plot(DirectPowerSpectrumTemplate(),
'direct', {'Omega_m': np.linspace(0.28, 0.32, n), 'logA': np.linspace(2.8, 3.2, n), 'h': np.linspace(0.65, 0.75, n)})

In [ ]:
from desilike.theories import Cosmoprimo

cosmo = Cosmoprimo(fiducial='DESI')
# Los parámetros default están específicados en formato .yaml, pero pueden ser actualizados:
print(cosmo.init.params)
# Cambios a cosmo.init.params impactan cualquier calculador que dependa de cosmo
# Changes a cosmo.all_params impactan sólo a este calculador

del cosmo.init.params['omega_cdm']
cosmo.init.params['Omega_m'] = { 
        'prior': {'limits': [0.01, 0.99]},
        'ref': {'dist': 'norm', 'loc': 0.3, 'scale': 0.01},
        'latex': r'\Omega_{m}',
        'delta': 0.005}
for name in ['n_s', 'tau_reio']:
    cosmo.init.params[name].update(fixed=True)
template = DirectPowerSpectrumTemplate(cosmo=cosmo, z=1.1)
print(template.all_params['Omega_m'].ref.loc)

Ver `desilike.theories.galaxy_clustering.power_template` para todos los templates.

## Interludio: Es hora de pedir ayuda!

In [ ]:
help(ShapeFitPowerSpectrumTemplate)

## Teoría

Actualmente disponibles (para la mayoría de ellos, la contraparte en función de correlación es cambiar PowerSpectrum por CorrelationFunction: ...PowerSpectrumMultipoles $\Rightarrow$ ...CorrelationFunctionMultipoles):

- Simple Kaiser model: KaiserTracerPowerSpectrumMultipoles

- Velocileptors: LPTVelocileptorsTracerPowerSpectrumMultipoles (plus other flavors)

- PyBird: PyBirdTracerPowerSpectrumMultipoles

- FOLPS: FOLPSTracerPowerSpectrumMultipoles

- TNS 1 loop: TNSTracerPowerSpectrumMultipoles

- empirical BAO: DampedBAOWigglesPowerSpectrumMultipoles

- PNG (scale-dependent bias): PNGTracerPowerSpectrumMultipoles

- ¡Ingrese su código propio aquí!

In [ ]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, LPTVelocileptorsTracerPowerSpectrumMultipoles, PyBirdTracerPowerSpectrumMultipoles

# O LPTVelocileptorsTracerPowerSpectrumMultipoles, PyBirdTracerPowerSpectrumMultipoles, etc.
theory = KaiserTracerPowerSpectrumMultipoles(template=template)
# Esta es la manera en que cualquier calculador (template, teoría, etc.) puede ser llamado
power = theory(b1=1.8)
fig, ax = plt.subplots(1, 1, sharex=False, sharey=True, figsize=(5, 3), squeeze=True)
for ill, ell in enumerate(theory.ells):
    ax.plot(theory.k, theory.k * theory.power[ill], label=r'$\ell = {:d}$'.format(ell))
ax.legend()
ax.set_xlabel(r'$k$ [$h/\mathrm{Mpc}$]')
ax.set_ylabel(r'$k P_{\ell}(k)$ [$(\mathrm{Mpc}/h)^{2}$]');

For correlation function, change "PowerSpectrumMultipoles" for "CorrelationFunctionMultipoles".

In [ ]:
from desilike.theories.galaxy_clustering import KaiserTracerCorrelationFunctionMultipoles, LPTVelocileptorsTracerCorrelationFunctionMultipoles, PyBirdTracerCorrelationFunctionMultipoles

# Or PyBirdTracerCorrelationFunctionMultipoles, LPTVelocileptorsTracerCorrelationFunctionMultipoles, etc.
theory = KaiserTracerCorrelationFunctionMultipoles(template=template)
# This is how to get all varied parameters
print(theory.varied_params)
corr = theory(b1=1.8, h=0.69)
fig, ax = plt.subplots(1, 1, sharex=False, sharey=True, figsize=(5, 3), squeeze=True)
for ill, ell in enumerate(theory.ells):
    ax.plot(theory.s, theory.s**2 * corr[ill], label=r'$\ell = {:d}$'.format(ell))
ax.legend()
ax.set_xlabel(r'$s$ [$\mathrm{Mpc}/h$]')
ax.set_ylabel(r'$s^{2} \xi_{\ell}(s)$ [$(\mathrm{Mpc}/h)^{2}$]');

## Interludio: instala cualquier calculador! (y sampler y profiler)

In [ ]:
from desilike import setup_logging

setup_logging()  # set logging

from desilike import Installer

theory = LPTVelocileptorsTracerCorrelationFunctionMultipoles(template=template)
installer = Installer(user=True)
installer(theory)

## Observable

Entonces, queremos comparar la teoría con los datos (un *observable*), típicamente:

- multipolos del espectro de potencias, con `TracerPowerSpectrumMultipolesObservable`
- multipolos de la función de correlación, con `TracerCorrelationFunctionMultipolesObservable`

In [ ]:
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable, TracerCorrelationFunctionMultipolesObservable

template = ShapeFitPowerSpectrumTemplate(z=1.1)
b1 = 1.5
theory = KaiserTracerPowerSpectrumMultipoles(template=template)
observable = TracerPowerSpectrumMultipolesObservable(\
             data={'b1': b1},  # un (o lista de) (path a) objeto PowerSpectrumStatistics, array plano, o diccionario de parámetros
             covariance=None,  # un (o lista de) (path a) mock, array (matriz de covarianzas), o None
             klim={0: [0.01, 0.2, 0.005], 2: [0.01, 0.2, 0.005]},  # límites de k, entre 0.01 y 0.2 h/Mpc con pasos de 0.005 h/Mpc
             theory=theory)  # teoría definida previamente
# Ver ayuda: help(TracerPowerSpectrumMultipolesObservable)
# para más opciones (para proveer ells, k en lugar de klim, shotnoise...)
# además, https://github.com/cosmodesi/desilike/blob/main/nb/bao_examples.ipynb

Nota: Aún no tenemos una matriz de covarianzas, a calcularla (Gaussiana y sin aproximación de ventana)

In [ ]:
from desilike.observables.galaxy_clustering import BoxFootprint, ObservablesCovarianceMatrix

footprint = BoxFootprint(volume=5e9, nbar=1e-4)  # caja con volumen de 5 (Gpc/h)^3 y densidad de 1e-4 (h/Mpc)^3
covariance = ObservablesCovarianceMatrix(observables=[observable], footprints=[footprint])
cov = covariance(b1=b1)   # evaluar matriz de covarianza con este parámetro

## Likelihood

In [ ]:
from desilike.likelihoods import ObservablesGaussianLikelihood

# No hay necesidad de especificar la covarianza si ya fue definida en el observable (TracerPowerSpectrumMultipolesObservable)
# Si hay mocks dados para cada observable, la matriz de covarianza del likelihood se computa sobre la marcha,
# usando mocks de cada observable (tomando en cuenta correlaciones)
likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov)
print(likelihood(b1=b1))  # en realidad logposterior = likelihood.loglikelihood + likelihood.logprior

In [ ]:
observable.plot_covariance_matrix();

In [ ]:
observable.plot();

La likelihood (o cualquier otro calculador) pueden ser llamados en cualquier punto con

In [ ]:
likelihood(b1=1., sn0=1000.)  # Actualizar bias lineal b1, y shot noise sn0
likelihood(qpar=0.99)  #  Actualizar parámetro de escala qpar; b1 y sn0 se mantienen fijos en 1. y 1000.
likelihood(sn0=100.)  # Actualizar shot noise, el remplate debe ser re-calculado

theory.power  # Contiene multipolos del espectro de potencias, evaluado en b1=1., qpar=0.99 y sn0=100.
theory(sn0=1000.);  # Recomputa la teoría en sn0=1000.

La likelihood (o cualquier otro calculador) pueden ser reparametrizados siguiendo, por ejemplo;

In [ ]:
likelihood.all_params['qpar'].update(derived='{qiso} * {qap}**(2. / 3.)')
likelihood.all_params['qper'].update(derived='{qiso} * {qap}**(- 1. / 3.)')
# Luego agregar qiso, qap a la colección de parámetros
likelihood.all_params['qiso'] = {'prior': {'limits': [0.9, 1.1]}, 'latex': 'q_{\mathrm{iso}}'}
likelihood.all_params['qap'] = {'prior': {'limits': [0.9, 1.1]}, 'latex': 'q_{\mathrm{ap}}'}
# Ahora la likelihood tiene como parámetros a variar
likelihood.varied_params  # b1, sn0, df, dm, qiso, qap

Es posible actualizar cualquier calculador con `calculator.init.update`

In [ ]:
template = DirectPowerSpectrumTemplate(z=1.1)
theory.init.update(template=template)  # Esto impacta cualquier calculador que dependa en teoría
likelihood.varied_params

## Combinando observables

In [ ]:
theory_xi = KaiserTracerCorrelationFunctionMultipoles(template=template)
observable_xi = TracerCorrelationFunctionMultipolesObservable(\
                data={'b1': b1},
                covariance=None,
                slim={0: [20., 150., 5.], 2: [20., 150., 5.]},
                theory=theory_xi)

In [ ]:
covariance_pk_xi = ObservablesCovarianceMatrix(\
                   observables=[observable, observable_xi],
                   footprints=footprint)
likelihood = ObservablesGaussianLikelihood(\
                    observables=[observable, observable_xi],
                    covariance=covariance_pk_xi(b1=b1))
likelihood()
likelihood.plot_covariance_matrix();

## Sumando likelihoods

In [ ]:
theory2 = theory.deepcopy()
for name in ['b1', 'sn0']:
    theory2.init.params[name].update(namespace='tracer2')
observable2 = TracerPowerSpectrumMultipolesObservable(\
             data={'tracer2.b1': 2.},
             covariance=covariance(b1=2),
             klim={0: [0.01, 0.2, 0.005], 2: [0.01, 0.2, 0.005]},
             theory=theory2)
likelihood2 = ObservablesGaussianLikelihood(observables=observable2)
like = likelihood + likelihood2
like.varied_params

La likelihood puede ser marginalizada analíticamente sobre los parámetros lineales (aquí `sn0`, `tracer2.sn0`)

In [ ]:
# '.best': establecer sn0 en su best fit
# '.marg': marginalización, asumiendo likelihood Gaussiana
# '.auto': automáticamente elegir entre '.best' (likelihood profiling) y '.marg' (likelihood sampling)
for param in like.all_params.select(name='*sn0'):
    param.update(derived='.auto')
# Ahora la likelihood tiene para varios parámetros variados (no sn0)
like.varied_params

## Parámetros comprimidos
Interpretación de los parámetros comprimidos (por ejemplo, $q_{\parallel}$, $q_{\perp}$, $df$, $dm$) en inferencia de parámetros cosmológicos.

In [ ]:
# data temporal y covarianza: estos pueden ser computados con samplers o herramientas Fishers con wrappers en desilike
from desilike.observables.galaxy_clustering import BAOCompressionObservable,\
StandardCompressionObservable, ShapeFitCompressionObservable

observable = ShapeFitCompressionObservable(data=[1., 1., 0., 1.],
                                           covariance=np.diag([0.01, 0.01, 0.0001, 0.01]),
                                           quantities=['qpar', 'qper', 'dm', 'df'], z=1.1)
likelihood = ObservablesGaussianLikelihood(observables=[observable])
print(likelihood.varied_params)

# Bindings

Ahora tenemos nuestro likelihood, podemos hacer un binding con un código de inferencia cosmológica externo (Cobaya, CosmoSIS y MontePython).

Recapitulemos la definición de la likelihood en una función

In [ ]:
def MyLikelihood():

    from desilike.theories.galaxy_clustering import DirectPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles
    from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable, BoxFootprint, ObservablesCovarianceMatrix
    from desilike.likelihoods import ObservablesGaussianLikelihood

    # 'external' significa "obtiene cantidades primordiales de un código externo, como cobaya"
    template = DirectPowerSpectrumTemplate(z=1.1, cosmo='external')
    theory = KaiserTracerPowerSpectrumMultipoles(template=template)
    observable = TracerPowerSpectrumMultipolesObservable(data={'b1': 1.2}, covariance=None,
                                                         klim={0: [0.01, 0.2, 0.005], 2: [0.01, 0.2, 0.005]}, theory=theory)
    footprint = BoxFootprint(volume=1e9, nbar=1e-3)
    covariance = ObservablesCovarianceMatrix(observables=observable, footprints=footprint)
    cov = covariance(b1=1.2)
    return ObservablesGaussianLikelihood(observables=observable, covariance=cov)

Pasemos esta función a los generadores que escribirán los archivos necesarios
para importarla como una likelihood externa en Cobaya, CosmoSIS y Montepython

Se puede dar `name_like` para asignarle un nombre a la likelihood  que sea distinto a `MyLikelihood`
y `kw_like` para pasar argumentos opcionales a `MyLikelihood`.

In [ ]:
from desilike.bindings import CobayaLikelihoodGenerator, CosmoSISLikelihoodGenerator, MontePythonLikelihoodGenerator
## NOTAAAAA: Esto solo jala en archivos .py
CobayaLikelihoodGenerator()(MyLikelihood)
CosmoSISLikelihoodGenerator()(MyLikelihood)
MontePythonLikelihoodGenerator()(MyLikelihood)

# Emuladores

Modelos de teoría EFT típicamente toman ~ 1 second en corren. Proveemos herramientas para emulación:

- Expansión de Taylor, hasta un orden dado, con `TaylorEmulatorEngine`
- Red Neural (perceptron de capas múltiples), con `MLPEmulatorEngine`

In [ ]:
template = DirectPowerSpectrumTemplate(z=1.1)
template.init.params['omega_b'].update(fixed=True)
theory = KaiserTracerPowerSpectrumMultipoles(template=template)
observable = TracerPowerSpectrumMultipolesObservable(data={'b1': 1.8}, covariance=None,
                                                     klim={0: [0.01, 0.2, 0.005], 2: [0.01, 0.2, 0.005]}, theory=theory)
footprint = BoxFootprint(volume=1e9, nbar=1e-3)
covariance = ObservablesCovarianceMatrix(observables=observable, footprints=footprint)
likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=covariance(b1=1.8))

Emulemor la parte de Teoría de Perturbación (.pt)
haciendo una expansión de Taylos de 2do orden (excepto por sn0)
Las derivadas de la teoría son evaluadas con auto-diferenciación de jax si es posible,
de otra manera, con diferencias finitas.

In [ ]:
from desilike.emulators import Emulator, TaylorEmulatorEngine, EmulatedCalculator

emulator = Emulator(theory.pt,
                    engine=TaylorEmulatorEngine(order={'*': 2, 'sn0': 1}))
emulator.set_samples()
emulator.fit()  # Definir expansión de Taylor

# El emulador puede ser guardado con:
emulator.save('_tests/emulator.npy')
# Y recargado con:
pt = EmulatedCalculator.load('_tests/emulator.npy')

theory.init.update(pt=pt)
# Ahora la teoría se ejecutará mucho más rápido!
theory(logA=3.);

# Profilers, samplers

Ya que queremos probar inferencia cosmológica en tiempo real
(sin recurrir a montepython, cosmosis o cobaya),
proveemos un wrapper para algunos profilers y samplers.

Profilers actualmente disponibles:
- [minuit](https://github.com/scikit-hep/iminuit), usado por la comunidad de física de altas energías, con `MinuitProfiler`
- [bobyqa](https://github.com/numericalalgorithmsgroup/pybobyqa), con `BOBYQAProfiler`
- [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize>), con `ScipyProfiler`

Estos pueden ser utilizados con, por ejemplo;

In [ ]:
from desilike.profilers import MinuitProfiler

profiler = MinuitProfiler(likelihood)
profiles = profiler.maximize(niterations=5)
profiles = profiler.interval(params=['h', 'Omega_m', 'logA'])
# Para asegurar los parámetros del mejor fit (de entre todas las iteraciones)
profiles.bestfit.choice()
# Para imprimer la información relevante
print(profiles.to_stats(tablefmt='pretty'))

## Recordatorio: installer

In [ ]:
installer = Installer(user=True)
installer(MinuitProfiler)

Samplers actualmente disponibles:
- [MCMC Sampler de Antony Lewis ](https://github.com/CobayaSampler/cobaya/tree/master/cobaya/samplers/mcmc>) con `MCMCSampler`
- [emcee ensemble sampler](https://github.com/dfm/emcee), con `EmceeSampler`
- [zeus ensemble slicing sampler](https://github.com/minaskar/zeus), con `ZeusSampler`
- [pocomc sampler precondicionado Monte-Carlo](https://github.com/minaskar/pocomc), con `PocoMCSampler`
- [dynesty nested sampler](https://github.com/joshspeagle/dynesty), con `DynamicDynestySampler`
- [polychord nested sampler](https://github.com/PolyChord/PolyChordLite), con `NestedSampler`

Esto puede ser usar con, por ejemplo;

In [ ]:
from desilike.samplers import EmceeSampler
from desilike.samples import Chain

sampler = EmceeSampler(likelihood, seed=42, chains=1)
chains = sampler.run(check={'max_eigen_gr': 0.05})  # correr hasta alcanzar el criterio Gelman-Rubin < 0.05
chain = Chain.concatenate([chain.remove_burnin(0.5) for chain in chains])[::10]  # adelgazamiento

In [ ]:
# Para imprimir información relevante
print(chain.to_stats(tablefmt='pretty'))

In [ ]:
from desilike.samples import plotting
plotting.plot_triangle(chain, params=['h', 'Omega_m', 'logA'])

# Fisher

Proveemos una rutina para estimación de Fisher

In [ ]:
from desilike import Fisher

fisher = Fisher(likelihood)
# Estimar una matriz [precisa] de Fisher con b1=2, usando auto-diferenciación de jax donde sea posible
# de otra manera, diferencias finitas (con pasos tamaño :attr:`Parameter.delta`)
fish = fisher(b1=2.)

In [ ]:
# LikelihoodFisher puede ser guardada con:
fish.save('_tests/fisher.npy')
# Y recargada con:
from desilike import LikelihoodFisher
fish = LikelihoodFisher.load('_tests/fisher.npy')

# Interludio: MPI

- profilers (iteraciones, empezar desde distintos puntos, en paralelo)
- samplers (chains y paralelización para ensemble samplers)
- emuladores (teoría evaluada en paralelo)
- fisher (derivadas en paralelo)
todos se benefician de MPI. Sólo es necesario correr el script con:

```mpiexec -np 64 python yourscript.py```

En NERSC:

```srun -n 64 python yourscript.py```

Si no se han aburrido, ahí les van unos links a ejemplos reales!

[Aplicación al mock challenge](https://github.com/cosmodesi/desi-y1-kp7/tree/main/mock_challenge_y1)

[Aplicación a los mocks de Y1 para inferencia cosmológica](https://github.com/cosmodesi/desi-y1-kp7/tree/main/mock_y1): forecasts directos y comprimidos

# Recapitulación

desilike es una estructura que (con un poco de suerte) hace fácil definir likelihoods de clustering, y provee wrappers para muchar utilidades útiles a la hora de hacer inferencia: emuladores, profiles, samplers, estimación Fisher.

[desilike docs](https://desilike.readthedocs.io/), en particular [getting started](https://desilike.readthedocs.io/en/latest/user/getting_started.html)

[desilike notebooks](https://github.com/cosmodesi/desilike/tree/main/nb)

Environment en NERSC que incluye desilike, Cobaya, CosmoSIS, Class, camb, y varios samplers instalados:
```
source /global/common/software/desi/users/adematti/cosmodesi_environment.sh main  # source the environment
# Para configurar el kernel correspondiente para uso en jupyter notebooks
${COSMODESIMODULES}/install_jupyter_kernel.sh main  # Este paso sólo es necesario hacerlo una vez
```